In [1]:
import os
os.chdir( "/home/lucasfuzato/TCC/CODE" )
from itertools import product
from dataclasses import make_dataclass

import pandas as pd
import numpy as np

from src.util.aux_fun import *
from src.util.constantes import *

/home/lucasfuzato/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# this dataframe will contain basic information of
# the simulations that will be used by the first network
# starting by the initial conditions
cond_df = pd.DataFrame( columns = [ "theta_0" , "v_0"] )

# initial condition will be given by launch angle in degrees
# ( theta_0 ) , and modular velocity in meters/sec ( v_0 )
prod = product( launch_angles , initial_velocities )
for it , ( theta_0 , v_0 ) in enumerate( prod ):
    cond_df.at[ it , "theta_0" ] = theta_0
    cond_df.at[ it , "v_0" ] = v_0

# for good measure
print( cond_df.head(10) )

  theta_0 v_0
0      15  50
1      15  52
2      15  54
3      15  56
4      15  58
5      15  60
6      15  62
7      15  64
8      15  66
9      15  68


In [3]:
# getting the train test split, every 5th simulation
# will be in the test_set and the rest will be in the
# training one

# splitting positions
split_pos = 5
arr = np.array( [False]*len( cond_df ) )
arr[ ::split_pos ] = True

# adding to the dataframe
cond_df[ "test_set" ] = pd.Series( arr )

# and printing for good measure
print( cond_df.head(15) )

# grouping for check if the mean values for the diferent
# conditions are very diferent from each other
cond_groups = cond_df.groupby( by = [ "test_set" ] )
print( "\n mean values ----------" )
print( cond_groups.mean() )

print( "\n standard deviation ----------" )
print( cond_groups.std() )

   theta_0 v_0  test_set
0       15  50      True
1       15  52     False
2       15  54     False
3       15  56     False
4       15  58     False
5       15  60      True
6       15  62     False
7       15  64     False
8       15  66     False
9       15  68     False
10      15  70      True
11      15  72     False
12      15  74     False
13      15  76     False
14      15  78     False

 mean values ----------
            theta_0        v_0
test_set                      
False     37.020921  74.958159
True      36.916667  75.166667

 standard deviation ----------
            theta_0        v_0
test_set                      
False     13.269327  15.001688
True      13.365944  15.117653


In [4]:
# -----------------------------------------------
# adding some extra columns
#
# real_time -> time it took to the simulation to be completed
# sim_time -> simulated trajectory time
# max_y , max_x -> self.explanatory
cond_df[ ["real_time" , "sim_time" , "max_x" , "max_y"] ] = np.nan
print( cond_df.head( 10 ))

  theta_0 v_0  test_set  real_time  sim_time  max_x  max_y
0      15  50      True        NaN       NaN    NaN    NaN
1      15  52     False        NaN       NaN    NaN    NaN
2      15  54     False        NaN       NaN    NaN    NaN
3      15  56     False        NaN       NaN    NaN    NaN
4      15  58     False        NaN       NaN    NaN    NaN
5      15  60      True        NaN       NaN    NaN    NaN
6      15  62     False        NaN       NaN    NaN    NaN
7      15  64     False        NaN       NaN    NaN    NaN
8      15  66     False        NaN       NaN    NaN    NaN
9      15  68     False        NaN       NaN    NaN    NaN


In [5]:
# dataframe with the trajectories of the launched objects
# each entry will have a fk to the initial condition, a timestamp,
# and the position ( x , y ) on the given timestamp
columns = [( "simu_id", int ) , "t" , "x" , "y" ]
row  = make_dataclass( "row", columns ) 
ser_lst = []
# tr_df = pd.DataFrame( columns = columns )

# amount of iterations of the rk process
# to make one record on the trajectort data
# frame
num_iter = int( R_STEP/H_STEP )

for i in cond_df.index:

    # getting the initial state for the simulation
    # each state handled by the rk machine is in the
    # format [ x , y , vx , vy ], and every initial state
    # has x and y equal to zero
    theta_0 = cond_df.at[ i , "theta_0" ]
    theta_rad = np.deg2rad( theta_0 )    # numpy works with angles in radian format

    v_0 = cond_df.at[ i , "v_0" ]
    vx = v_0*np.cos( theta_rad )
    vy = v_0*np.sin( theta_rad )

    # initial state, rk machine that will work with it
    # and the iterator.
    state_0 = np.array( [ 0 , 0 , vx , vy ] )
    rk = get_rk4_machine( state_0 , H_STEP , state_update )
    rk_iter = rk_iterator( rk , num_iter = num_iter )

    # now making the simulation
    while True:

        # # entry corresponding to the trajectory point
        # ser = pd.Series( index = columns )

        # assigning initial condition
        simu_id = i

        # getting the next state 
        t , state = next( rk_iter )
        x = state[ 0 ]
        y = state[ 1 ]

        # assigning to the list
        ser_lst.append( row( simu_id , t , x , y ) )

        # simulation ends when the ball hits the 
        # ground
        if y < 0:
            break
    
    # getting the real time it took
    # to make the simulation
    cond_df.at[ i , "real_time" ] = np.round( 1000*rk.clock , 2 )

# finally making the dataframe
tr_df = pd.DataFrame( ser_lst )
print( tr_df.head() )



   simu_id      t         x         y
0        0  0.000  0.000000  0.000000
1        0  0.005  0.240900  0.064541
2        0  0.010  0.480645  0.128753
3        0  0.015  0.719246  0.192639
4        0  0.020  0.956714  0.256198


In [6]:
# now, time to find de max values of the simulation time
# x and y for each simulation
tr_max = tr_df.groupby( by = "simu_id" ).max()
cond_df[ "sim_time" ] = tr_max[ "t" ]
cond_df[ "max_x" ] = tr_max[ "x" ]
cond_df[ "max_y" ] = tr_max[ "y" ]

print( cond_df.head() )

  theta_0 v_0  test_set  real_time  sim_time      max_x     max_y
0      15  50      True    1171.61     2.450  60.693953  7.349777
1      15  52     False    1169.18     2.535  63.299003  7.864067
2      15  54     False    1224.86     2.620  65.863071  8.387537
3      15  56     False    1283.68     2.700  68.316822  8.919449
4      15  58     False    1294.20     2.780  70.730941  9.458993


In [7]:
# finally saving
cond_df.to_csv( COND_FILE , index = False )
tr_df.to_csv( TR_FILE , index = False)

In [8]:
print( AIR_DRAG )

0.02
